** In this Jupyter Notebook, the absolute + sensitivity analysis results are generated for a One-at-a-time sensitivity analysis of cooling time for the proliferation risk evaluation metric. The reason why this metric is separated from the others is that you need the -exp version of each sqlite and pyne on your machine to make it work **

In [2]:
import cymetric as cym
from cymetric import timeseries
import matplotlib as plt 
import pandas as pd
import numpy as np
import sys 
sys.path.insert(0, '../../../scripts/')
import output as oup

/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.enrichment is not yet QA compliant.
  return f(*args, **kwds)


# This is when you already have the results 

In [7]:
df_p = pd.read_csv('ct-df-pu.csv',index_col='CT')
df_p

,Max Pu in all CP,Pu Quality in all CP at Max Pu,Max Pu in HLW,Pu Quality in HLW at Max Pu,Max Pu in all RPR,Pu Quality in all RPR at Max Pu
CT,,,,,,
0,48979.28149,0.658285,44426.64870,0.620082,2800580.691,0.490396
2,199036.05600,0.604012,43629.18450,0.619566,2637125.071,0.483398
4,368405.04070,0.583823,42844.40218,0.618849,2460403.988,0.478289
6,554321.65720,0.570506,42646.71544,0.618617,2290857.918,0.473204
8,733483.85210,0.560343,42858.09938,0.618184,2111786.283,0.468897


# The rest of the code below is to generate the above results

In [ ]:
starter_string = 'CT'
scenario_nums = ['0','2','4','6','8']

In [ ]:
df_p = oup.initialize_df(scenario_index=starter_string,
                                 scenarios_nums=scenario_nums)

In [ ]:
df_p['Max Pu in all CP'] = 0
df_p['Pu Quality in all CP at Max Pu'] = 0
df_p['Max Pu in HLW'] = 0
df_p['Pu Quality in HLW at Max Pu'] = 0
df_p['Max Pu in all RPR'] = 0
df_p['Pu Quality in all RPR at Max Pu'] = 0

In [ ]:
output_start = '../cyclus-files/oat/cooling-time/ct'
ev_dict = {}
for x in range(len(scenario_nums)): 
    output_file = output_start + scenario_nums[x]+'-exp.sqlite'
    ev_dict[scenario_nums[x]] = cym.Evaluator(db=cym.dbopen(output_file),write=True)

In [ ]:
for x in range(len(scenario_nums)): 
    cp = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrstorage','moxstorage','frstorage'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity']
    fissile_cp = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrstorage','moxstorage','frstorage'],nucs=['pu-239','pu-241'])['Quantity']
    df_p.loc[scenario_nums[x],'Max Pu in all CP'] = cp.max()
    df_p.loc[scenario_nums[x],'Pu Quality in all CP at Max Pu'] = fissile_cp[cp.idxmax()]/cp.max()
    hlw = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['enrichmentsink','lwrsink','moxsink','frsink'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity']
    fissile_hlw = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['enrichmentsink','lwrsink','moxsink','frsink'],nucs=['pu-239','pu-241'])['Quantity']
    df_p.loc[scenario_nums[x],'Max Pu in HLW'] = hlw.max()
    df_p.loc[scenario_nums[x],'Pu Quality in HLW at Max Pu'] = fissile_hlw[hlw.idxmax()]/hlw.max()
    rpr = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrreprocessing','moxreprocessing','frreprocessing'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity']
    fissile_rpr = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrreprocessing','moxreprocessing','frreprocessing'],nucs=['pu-239','pu-241'])['Quantity']
    df_p.loc[scenario_nums[x],'Max Pu in all RPR'] = rpr.max()
    df_p.loc[scenario_nums[x],'Pu Quality in all RPR at Max Pu'] = fissile_rpr[rpr.idxmax()]/rpr.max()

In [ ]:
df_p.to_csv('ct-df-pu.csv')

In [8]:
df_p_sa = oup.sensitivity(2,df_p)
df_p_sa

,Max Pu in all CP,Pu Quality in all CP at Max Pu,Max Pu in HLW,Pu Quality in HLW at Max Pu,Max Pu in all RPR,Pu Quality in all RPR at Max Pu
CT,,,,,,
0,-75.391754,8.985512,1.827823,0.083333,6.198251,1.447639
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,85.094625,-3.342451,-1.798755,-0.115688,-6.701278,-1.056889
6,178.503136,-5.547091,-2.251862,-0.153037,-13.130479,-2.108881
8,268.518080,-7.229720,-1.767361,-0.223029,-19.920890,-2.999790


In [9]:
df_p_sa.to_csv('ct-df-pu-sa.csv')